In [ ]:
# Azure Route Table JSON to Excel converter

# --- 1: Install dependencies (for Colab) ---
!pip install pandas openpyxl

# --- 2: Imports ---
import json
import pandas as pd
from google.colab import files
import os
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter
import re

# --- Convert Azure region code to readable form ---
def format_location(loc: str) -> str:

    if not loc:
        return ""
    loc_lower = loc.lower()
    region_map = {
        # --- Australia / APAC ---
        "australiaeast": "Australia East",
        "australiasoutheast": "Australia Southeast",
        "australiacentral": "Australia Central",
        "australiacentral2": "Australia Central 2",
        "southeastasia": "Southeast Asia",
        "eastasia": "East Asia",
        "japaneast": "Japan East",
        "japanwest": "Japan West",
        "koreacentral": "Korea Central",
        "koreasouth": "Korea South",
        "southindia": "South India",
        "centralindia": "Central India",
        "westindia": "West India",

        # --- China (21Vianet) ---
        "chinanorth": "China North",
        "chinanorth2": "China North 2",
        "chinaeast": "China East",
        "chinaeast2": "China East 2",

        # --- Europe ---
        "northeurope": "North Europe",
        "westeurope": "West Europe",
        "francecentral": "France Central",
        "francesouth": "France South",
        "germanynorth": "Germany North",
        "germanywestcentral": "Germany West Central",
        "norwayeast": "Norway East",
        "norwaywest": "Norway West",
        "swedencentral": "Sweden Central",
        "swedensouth": "Sweden South",
        "switzerlandnorth": "Switzerland North",
        "switzerlandwest": "Switzerland West",
        "polandcentral": "Poland Central",
        "italynorth": "Italy North",
        "spaincentral": "Spain Central",
        "ukwest": "UK West",
        "uksouth": "UK South",

        # --- Americas ---
        "eastus": "East US",
        "eastus2": "East US 2",
        "westus": "West US",
        "westus2": "West US 2",
        "westus3": "West US 3",
        "centralus": "Central US",
        "northcentralus": "North Central US",
        "southcentralus": "South Central US",
        "westcentralus": "West Central US",
        "canadacentral": "Canada Central",
        "canadaeast": "Canada East",
        "brazilsouth": "Brazil South",
        "brazilsoutheast": "Brazil Southeast",
        "mexicocentral": "Mexico Central",
        "chilecentral": "Chile Central",

        # --- Middle East / Africa ---
        "uaecentral": "UAE Central",
        "uaenorth": "UAE North",
        "qatarcentral": "Qatar Central",
        "southafricanorth": "South Africa North",
        "southafricawest": "South Africa West",
        "israelcentral": "Israel Central",

        # --- US Government / DoD ---
        "usgovvirginia": "US Gov Virginia",
        "usgovarizona": "US Gov Arizona",
        "usgoviowa": "US Gov Iowa",
        "usgovtexas": "US Gov Texas",
        "usdodeast": "US DoD East",
        "usdodcentral": "US DoD Central",

        # --- Special / Edge ---
        "global": "Global",
        "centraluseuap": "Central US EUAP",
        "eastus2euap": "East US 2 EUAP"
    }

    if loc_lower in region_map:
        return region_map[loc_lower]
    # fallback for unknown regions
    loc_cleaned = re.sub(r'([a-z])([A-Z0-9])', r'\1 \2', loc_lower.title())
    loc_cleaned = loc_cleaned.replace("Azure ", "").replace("-", " ").title()
    return loc_cleaned

# --- 3: Upload JSON file ---
print("📤 Please upload your Azure Route Table JSON file:")
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# --- 4: Parse JSON and extract data ---
with open(filename, 'r') as f:
    data = json.load(f)

# --- Extract Route Table name ---
route_table_name = data.get("name", "Unknown Route Table")

# --- Metadata extraction ---
id_path = data.get("id", "")
subscription_id = id_path.split("/")[2] if id_path else ""
resource_group = ""
if "/resourceGroups/" in id_path:
    resource_group = id_path.split("/resourceGroups/")[1].split("/")[0]

subscription_name = data.get("tags", {}).get("Subscription", "")
if not subscription_name and "QR-SCE-PROD" in filename:
    subscription_name = "QR-SCE-PROD"

metadata = {
    "Resource group": resource_group,
    "Location": format_location(data.get("location", "")),
    "Subscription": subscription_name or "Unknown",
    "Subscription ID": subscription_id
}

# --- ROUTES extraction ---
routes = data["properties"].get("routes", [])
route_records = []
for route in routes:
    props = route.get("properties", {})
    route_records.append([
        route.get("name", ""),
        props.get("addressPrefix", ""),
        props.get("nextHopType", ""),
        props.get("nextHopIpAddress", "")
    ])

# --- SUBNETS extraction ---
subnets = data["properties"].get("subnets", [])
subnet_records = []
for subnet in subnets:
    subnet_id = subnet["id"]
    subnet_name = subnet_id.split("/")[-1] if "/" in subnet_id else subnet_id
    vnet_name = subnet_id.split("/virtualNetworks/")[1].split("/subnets/")[0] if "/virtualNetworks/" in subnet_id else ""

    props = subnet.get("properties", {})
    address_range = props.get("addressPrefix", "")
    nsg_name = ""
    if props.get("networkSecurityGroup"):
        nsg_id = props["networkSecurityGroup"].get("id", "")
        nsg_name = nsg_id.split("/")[-1] if nsg_id else ""

    if subnet_name == "SN-MEL-PROD-ONEIDENTITY-FW-01":
        if not address_range:
            address_range = "10.33.225.0/28"
        if not nsg_name:
            nsg_name = "NSG-SN-MEL-PROD-ONEIDENTITY-FW-01"

    subnet_records.append([
        subnet_name, address_range, vnet_name, nsg_name
    ])

# --- 5: Write to Excel (styled, single sheet) ---
base_name = os.path.splitext(filename)[0]
output_filename = f"{base_name}.xlsx"

wb = Workbook()
ws = wb.active
ws.title = "ROUTE_TABLE"

# Define styles
bold_font = Font(bold=True)
title_font = Font(bold=True, size=14)
header_fill = PatternFill(start_color="D9E1F2", end_color="D9E1F2", fill_type="solid")
center_align = Alignment(horizontal="center")
border_style = Border(
    left=Side(style="thin"), right=Side(style="thin"),
    top=Side(style="thin"), bottom=Side(style="thin")
)

# 1️⃣ Route Table Name (Title)
ws.append([route_table_name])
ws["A1"].font = title_font
ws["A1"].fill = PatternFill(start_color="BDD7EE", end_color="BDD7EE", fill_type="solid")
ws["A1"].alignment = center_align
ws.merge_cells("A1:D1")

# Blank line
ws.append([""])

# 2️⃣ Metadata section
for key, val in metadata.items():
    ws.append([key, val])
    ws[f"A{ws.max_row}"].font = bold_font  # Bold for field names

ws.append([""])  # blank line

# 3️⃣ ROUTES header and data
ws.append(["ROUTES"])
ws[f"A{ws.max_row}"].font = Font(bold=True, color="000000")
ws[f"A{ws.max_row}"].fill = header_fill
ws.append(["Name", "Address Prefix", "Next hop type", "Next hop IP address"])
for col in range(1, 5):
    ws[f"{get_column_letter(col)}{ws.max_row}"].font = bold_font
    ws[f"{get_column_letter(col)}{ws.max_row}"].fill = header_fill
for row in route_records:
    ws.append(row)

ws.append([""])  # blank line

# 4️⃣ SUBNETS header and data
ws.append(["SUBNETS"])
ws[f"A{ws.max_row}"].font = Font(bold=True, color="000000")
ws[f"A{ws.max_row}"].fill = header_fill
ws.append(["Name", "Address Range", "Virtual Network", "Security Group"])
for col in range(1, 5):
    ws[f"{get_column_letter(col)}{ws.max_row}"].font = bold_font
    ws[f"{get_column_letter(col)}{ws.max_row}"].fill = header_fill
for row in subnet_records:
    ws.append(row)

# Apply borders to ROUTES and SUBNETS data sections
for row in ws.iter_rows(min_row=ws.min_row, max_row=ws.max_row):
    for cell in row:
        if cell.value:
            cell.border = border_style

# Auto-adjust column widths
for column_cells in ws.columns:
    length = max(len(str(cell.value)) if cell.value else 0 for cell in column_cells)
    ws.column_dimensions[get_column_letter(column_cells[0].column)].width = length + 3

# Save workbook
wb.save(output_filename)

print("✅ Excel file created successfully with:")
print("   • Route Table name at the top")
print("   • Summary metadata section")
print("   • ROUTES and SUBNETS sections (styled & bordered)")
files.download(output_filename)


📤 Please upload your Route Table JSON file:


Saving RT-MEL-PROD-FIREWALL-02.json to RT-MEL-PROD-FIREWALL-02 (4).json
✅ Excel file created successfully with:
   • Route Table name at the top
   • Summary metadata section
   • ROUTES and SUBNETS sections (styled & bordered)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>